In [2]:
from typing import List, Optional
from pydantic import BaseModel
from dotenv import load_dotenv

import chromadb
import openai
import instructor

from models import EssayAgentKnowledge, MathAgentKnowledge, Message, MessageRole

In [3]:
load_dotenv()

chroma_client = chromadb.Client()
openai_client = openai.OpenAI()
instructor_client = instructor.from_openai(openai_client)

In [4]:
class StudentAgent:
    def __init__(
        self, id: str, name: str, system_prompt: str, knowledge_cls: BaseModel
    ) -> None:
        self.id = id
        self.name = name
        self.system_prompt = system_prompt
        self.knowledge_cls = knowledge_cls
        self.knowledge_description = self._generate_knowledge_description()
        pass

    def respond(self, input: str, chat_history: List[Message]) -> Message:
        response = openai_client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=chat_history
            + [Message(role=MessageRole.user, content=input)],
        )

        return response.choices[0].message.content

    def get_knowledge(self):
        pass

    def _generate_knowledge_description(self) -> str:
        response = openai_client.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                Message(
                    role=MessageRole.system,
                    content="You are an expert in creating detailed, \
                        	human-readable descriptions(NOT JSON) for intelligent agents \
                            based on their specific system prompts and knowledge schemas. \
                            Your task is to generate a first-person description of the agent, \
                            explaining its purpose and capabilities. \
                            Additionally, you will describe the structured knowledge the agent \
                            stores according to the provided JSON schema. \
                            Ensure that the description is clear, comprehensive, and accessible to a general audience.",
                ),
                Message(
                    role=MessageRole.user,
                    content=(
                        f"Agent name: {self.name}\n\n"
                        f"Agent System prompt: {self.system_prompt}\n\n"
                        f"JSON Schema for Knowledge stored: {self.knowledge_cls.model_json_schema()}"
                    ),
                ),
            ],
        )

        return response.choices[0].message.content

In [5]:
EssayFeedbackAgent = StudentAgent(id="1", name="EssayFeedbackAgent", system_prompt="You are an expert at providing feedback on essays for 8th graders.", knowledge_cls=EssayAgentKnowledge)

In [90]:
print(EssayFeedbackAgent.respond(input="How do i write a good essay?", chat_history=[]))

Writing a good essay involves careful planning, clear organization, and effective communication of your ideas. Here are some tips to help you write a successful essay:

1. Understand the assignment: Before you start writing, make sure you fully understand the prompt or question you are being asked to address. Take the time to analyze the assignment and identify key requirements such as the topic, length, and format.

2. Research the topic: Conduct thorough research on the topic you are writing about. Gather relevant information from credible sources such as books, articles, and scholarly journals. Take notes and make sure to properly cite any sources you use in your essay.

3. Develop a thesis statement: A thesis statement is a concise statement that summarizes the main point or argument of your essay. Make sure your thesis is clear, specific, and arguable.

4. Create an outline: Organize your ideas and information by creating an outline for your essay. An outline will help you structu

In [44]:
print(EssayFeedbackAgent.knowledge_description)

Hello! I am the EssayFeedbackAgent, designed specifically to assist with providing thorough and constructive feedback on essays written by 8th graders. My main purpose is to help students enhance their writing skills by offering insights into various aspects of their essays, from the structure and clarity of their arguments to the persuasiveness and relevance of their evidence.

Let’s delve into my capabilities and how I structure my feedback:

**Capabilities:**
1. **Detailed Analysis:** I am programmed to analyze essays based on defined criteria that focus on essential components of essay writing. This includes the introduction, structure, argumentation, evidence, and conclusion.
2. **Constructive Feedback:** For each part of the essay, I provide a balanced view highlighting strengths, weaknesses, and offering constructive suggestions. This approach is aimed at fostering students’ skills in critical thinking and self-improvement.
3. **Targeted Suggestions:** My feedback is tailored to

In [45]:
MathTutorAgent = StudentAgent(id="2", name="MathTutorAgent", system_prompt="You are an expert math tutor for 8th graders.", knowledge_cls=MathAgentKnowledge)

In [46]:
print(MathTutorAgent.knowledge_description)

Hello! I am MathTutorAgent, a specialized digital tutor designed to assist 8th graders in mastering mathematics. My purpose is to provide tailored educational support, helping students strengthen their math skills, clarify doubts, and overcome common misconceptions in various mathematical topics.

As a virtual tutor, my capabilities include delivering personalized lessons, offering practice exercises, and providing instant feedback to ensure students understand the concepts. I am equipped with a deep understanding of the 8th-grade math curriculum and am trained to adapt my teaching methods to fit each student's unique learning style and pace.

To perform effectively, I maintain a structured knowledge base as outlined in my schema, which systematically categorizes my expertise and resources on each math topic relevant for 8th graders. Let me break down what my knowledge schema entails:

1. **TopicKnowledge**: This is the core of my knowledge base. Each entry in this category is dedicate

In [47]:
agents = [EssayFeedbackAgent, MathTutorAgent]

In [48]:
import chromadb.utils.embedding_functions as embedding_functions
import os

openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key=os.environ.get("OPENAI_API_KEY"),
                model_name="text-embedding-3-small"
            )

In [49]:
COLLECTION_NAME = "agent_knowledge_collection"

if chroma_client.get_collection(name=COLLECTION_NAME):
	chroma_client.delete_collection(name=COLLECTION_NAME)

agent_knowledge_collection = chroma_client.get_or_create_collection(name=COLLECTION_NAME, embedding_function=openai_ef)

In [50]:
def embed_text(text: str):
	return openai_client.embeddings.create(model="text-embedding-3-small", input=text).data[0].embedding

In [51]:
agent_knowledge_collection.add(
	documents=[agent.knowledge_description for agent in agents],
	ids=[agent.id for agent in agents],
	embeddings=[embed_text(agent.knowledge_description) for agent in agents]
)

In [56]:
def get_relevant_agent(query: str):
	res = agent_knowledge_collection.query(
		query_texts=[query],
		n_results=1
	)

	res_agent = None

	if len(res['ids'][0]) == 0:
		return res_agent
	
	agent_id = res['ids'][0][0]

	for agent in agents:
		if agent.id == agent_id:
			res_agent = agent
			break
	
	return res_agent

In [55]:
class TeacherAgentThought(BaseModel):
	knowledge_needed: Optional[str]

In [75]:
class TeacherAgent:
	def __init__(self, id: str, name: str, system_prompt: str) -> None:
		self.id = id
		self.name = name
		self.system_prompt = system_prompt
	
	def respond(self, input: str, chat_history: List[Message]) -> Message:
		knowledge_needed = self._think(input=input, chat_history=chat_history)

		if not knowledge_needed:
			messages = chat_history + Message(role=MessageRole.user, content=input)
		else:
			agent = get_relevant_agent(query=knowledge_needed)
			knowledge = agent.get_knowledge()

			messages = chat_history + [
				Message(
					role=MessageRole.system,
					content=(
						"Use this knowledge in combination with the teacher's query"
						"to personalise your response and best help the teacher.\n\n"
						f"Knowledge: {knowledge}"
					)
				),
				Message(
					role=MessageRole.user,
					content=input
				)
			]
		
		response = openai_client.chat.completions.create(
			model="gpt-3.5-turbo",
			messages=messages
		)

		return response.choices[0].message

	def _think(self, input: str, chat_history: List[Message]) -> Optional[str]:
		response = instructor_client.chat.completions.create(
			model="gpt-3.5-turbo",
			response_model=TeacherAgentThought,
			messages=[
				Message(
					role=MessageRole.system,
					content=(
						"You are an expert at generating a thought to decide"
						"if you need knowledge from student facing AI agents"
						"about student performance"
						"to best respond to the teacher's query"
						"If you do not need knowledge about students then you will return None"
						"Otherwise you will describe the knowledge you need"
					)
				),
				Message(
					role=MessageRole.user,
					content=input
				)	
			]
		)

		return response.knowledge_needed

In [70]:
MathWorksheetGenerator = TeacherAgent(id="1", name="MathWorksheetGenerator", system_prompt="You are an expert at generating math worksheets for 8th grade")

In [71]:
MathWorksheetGenerator.respond(input="Generate a worksheet on trigonometry that targets students weaknesses", chat_history=[])

knowledge_needed='student performance in trigonometry'


In [73]:
MathWorksheetGenerator.respond(input="sup", chat_history=[])

knowledge_needed=None
